# 1. Import dependencies

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

# 2. Instantiate the model

In [2]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

Downloading: 100%|██████████| 39.0/39.0 [00:00<00:00, 13.3kB/s]
Downloading: 100%|██████████| 953/953 [00:00<00:00, 436kB/s]
Downloading: 100%|██████████| 872k/872k [00:00<00:00, 1.24MB/s]
Downloading: 100%|██████████| 112/112 [00:00<00:00, 35.9kB/s]
Downloading: 100%|██████████| 669M/669M [03:31<00:00, 3.17MB/s]


# 3. Encode and Calculate the Sentiment

In [11]:
token = tokenizer.encode('I loved this product but quality could have been better', return_tensors='pt')
result = model(token)
result.logits
int(torch.argmax(result.logits))+1

3

# 4. Collect Reviews

In [41]:
r = requests.get('https://www.tripadvisor.com.sg/Hotel_Review-g294265-d301577-Reviews-Swissotel_Merchant_Court_Singapore-Singapore.html')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*IRsGHoPm.*')
results = soup.find_all('q', {'class':regex})
reviews = [result.text for result in results]

In [33]:
reviews[3]

'Had a great time here. Durian dessert is a must have dishes here. The restaurant staff Amanda was very helpful, and attentive, making it a very enjoyable experience! Nice place for linch buffet. Highly recommended.'

# 5. Load Reviews and populate sentiment scores (1-5 rating)

In [35]:
import numpy as np
import pandas as pd

In [39]:
df = pd.DataFrame(np.array(reviews), columns=['review'])
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1


In [42]:
sentiment_score(df['review'].iloc[4])
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))
df

,review,sentiment
0,The food is awesome and we were attended by pe...,5
1,We really enjoyed our dinner experience today!...,5
2,I really enjoyed our last night’s dinner here....,5
3,Had a great time here. Durian dessert is a mus...,5
4,Food is good. Was served by Harry Zhu and Ain....,4


In [38]:
df['review'].iloc[4]

'Food is good. Was served by Harry Zhu and Ain. Both were very courtesy and polite. Very diligent. Ensure cleanliness by clearing our empty plates. Good ambience for dinning. I especially like the mentai sushi, salmon sashimi were fresh. Generally is awesome 👍👍👍'